# Apriori Algorithm
1. Finding the frequent 1-itemsets
2. Generating candidates for n-itemsets and pruning them
3. Calculating the support of the candidates and saving the frequent n-itemsets

In [1]:
class Apriori:    
    def __init__(self, records, relative_min_support):
        self.records = records
        self.relative_min_support = relative_min_support
        self.absolute_min_support = len(records)*relative_min_support
        
        self._mapping_records(records)
        self._get_l1_itemset()
        self._get_frequent_itemsets()
        
    def _mapping_records(self, records):
        # collecting unique items
        items = list(set(item for record in records for item in record))

        # mapping items to ids
        self.id_to_item = items
        self.item_to_id = {item: i for (i, item) in enumerate(items)}

        # mapping items in records to the ids
        mapped_records = []
        for record in records:
            record_items = []
            for item in record:
                record_items.append(self.item_to_id[item])
            mapped_records.append(tuple(sorted(record_items)))

        self.mapped_records = mapped_records
    
    def _get_l1_itemset(self):
        records = self.mapped_records
        max_id = len(self.id_to_item)-1
        absolute_min_support = self.absolute_min_support
        
        # calculating support of length-1 itemsets
        l1_items = {item_id: 0 for item_id in range(0, max_id + 1)}
        for record in records:
            for item in record:
                l1_items[item] = l1_items[item] + 1

        # frequent length-1 itemsets
        frequent_l1_items = {item: sup for item, sup in l1_items.items() if sup >= absolute_min_support}

        self.frequent_items = {(item,): sup for item, sup in frequent_l1_items.items()}
        
    def _get_frequent_itemsets(self):
        # sorting l1 key values (item ids) by ascending order
        candidates = [item for item in sorted(self.frequent_items)]

        while len(candidates) > 0:
            # generate new candidates (l_n itemsets -> l_n+1 itemsets)
            new_candidates = self._generate(candidates)
            
            # calculating the support
            # the itemsets need to be a tuple to be hashable for the dictionary
            candidate_support = {tuple(itemset): self._calculate_support(itemset) for itemset in new_candidates}
            
            # considering only the frequent itemsets
            frequent_candidates = dict(
                filter(lambda itemset_support: itemset_support[1] >= self.absolute_min_support, candidate_support.items()))
            
            # adding the new itemsets to the dict with frequent itemsets
            self.frequent_items.update(frequent_candidates)
            # replacing the l_n itemsets with the l_n+1 itemsets
            candidates = [itemset for itemset in frequent_candidates]
        
    def _calculate_support(self, items):
        records = self.mapped_records
        return sum(1 if all(item in record for item in items) else 0 for record in records)
    
    def _generate(self, items):
        new_items = []

        # generating
        for i in range(len(items) - 1):
            for k in range(i + 1, len(items)):
                if items[i][:-1] == items[k][:-1]:
                    new_items.append(tuple([item for item in items[i]] + [items[k][-1]]))
                                        
        pruned_new_items = []
        # pruning
        for item in new_items:
            if all((tuple(item[:i] + item[i + 1:]) in items) for i in range(len(item))):
                pruned_new_items.append(item)

        return pruned_new_items
    
    def recommend(self, items):
        item_ids = sorted(self.item_to_id[item] for item in items)

        # Filtering the frequent (n+1)-itemsets that contain the items we need a recommendation to.
        # We do not need to consider (>n+1)-itemsets as these contain only the items from the (n+1)-itemsets.
        # In this case I recommend another item only if it is in a frequent dataset.
        # In some context it might be the desired behaviour.
        frequent_itemsets_query = [record for record in self.frequent_items
                                              if len(record) == len(item_ids)+1 and all(item_id in record for item_id in item_ids)]

        # finding the maximum support of the (n+1)-itemsets
        supports = tuple(self.frequent_items[itemset] for itemset in frequent_itemsets_query)
        if len(supports) == 0:
            print("I cannot recommend any item with a")
            return [], 0
        max_support = max(supports)

        # filtering the (n+1)-itemsets with the maximum support
        recommendations = tuple(filter(lambda itemset: self.frequent_items[itemset] == max_support,
                                       frequent_itemsets_query))

        # recommending every item with the maximum support
        items = []
        for recommendation in recommendations:
                items.append([self.id_to_item[item_id] for item_id in recommendation 
                                  if item_id not in item_ids])
        
        confidence = self.frequent_items[recommendation] / self.frequent_items[tuple(item_ids)]
        return items, confidence

In [2]:
input_records = [["A", "B", "C"], ["A", "B"], ["A", "C"]]
apriori = Apriori(input_records, 0.4)
apriori.recommend(["A"])

([['B'], ['C']], 0.6666666666666666)

In [3]:
input_records = [["A", "B", "C"], ["A", "B"], ["A", "C"], ["A", "B"]]
apriori = Apriori(input_records, 0)
apriori.recommend(["A"])

([['B']], 0.75)

In [4]:
input_records = [["A", "B", "C"], ["C", "D"], ["B", "C"]]
apriori = Apriori(input_records, 0.4)
apriori.recommend(["A"])

I cannot recommend any item with a


([], 0)

## Findings
In the example below both C and D are recommended as they have the confidence. However, if we look at the data, C appears more often with A and B separately than D does.

The confidence of recommending $B$ to the item(s) $A$ is defined as $\text{support}(A \cup B)\,/\,\text{support}(A)$ which means that it only considers how many times $A \cup B$ and $A$ appears in the data. There can be cases like the one below when there are multiple options with the same confidence, but a subsets of one of them appears more often than the subsets of the other. Depending on the context, recommending $D$ might be not as good as recommending $C$. In these cases, we need another definition that considers the subsets with the recommended item.

In [5]:
input_records = [["A", "B", "C"], ["A", "B", "D"], ["B", "C"], ["A", "C"], ["A", "C"], ["B", "C"]]
apriori = Apriori(input_records, 0)
apriori.recommend(["A", "B"])

([['C'], ['D']], 0.5)